In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet

# Step 1: Prepare data
df_daily = df.groupby('order_date')['total_price'].sum().reset_index()
df_daily.columns = ['ds', 'y']

# # Step 2: Split into train and test
# forecast_days = 7  # change to 30 for 1-month forecast
# train = df_daily[:-forecast_days]
# test = df_daily[-forecast_days:]
# Step 2: Split into train and test
forecast_days = 7  # or 30 for monthly
train = df_daily[:-forecast_days]
test = df_daily[-forecast_days:].copy()

# Step 3: Fit the model
model = Prophet()
model.fit(train)

# 🛠 Fix: ensure future includes correct number of days
future = model.make_future_dataframe(periods=len(test), freq='D')
forecast = model.predict(future)

# Step 5: Align on dates for MAPE calculation
forecast.set_index('ds', inplace=True)
test.set_index('ds', inplace=True)
predicted = forecast.reindex(test.index)



# Step 6: Calculate MAPE
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape_value = mape(test['y'], predicted['yhat'])
print(f'MAPE: {mape_value:.2f}%')

# Step 7: Plot results
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
plt.plot(train['ds'], train['y'], label='Train')
plt.plot(test.index, test['y'], label='Actual')
plt.plot(predicted.index, predicted['yhat'], label='Forecast')
plt.title('Sales Forecast vs Actual')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.legend()
plt.tight_layout()
plt.show()